In [ ]:
!pip install faiss-cpu transformers datasets sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
# Load a sentence embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Example startup-related documents
documents = [
    "Startup India provides funding and tax benefits for new startups in India.",
    "Angel investors are individuals who invest in early-stage startups in exchange for equity.",
    "A pitch deck is a presentation that startups use to attract investors.",
    "The government offers startup grants through various schemes.",
    "Networking events connect entrepreneurs with investors and mentors."
]

In [ ]:
# Convert documents to vectors
doc_vectors = embed_model.encode(documents, convert_to_numpy=True)

In [ ]:
# Initialize FAISS index
dimension = doc_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_vectors)

In [ ]:
def retrieve_docs(query, top_k=2):
    query_vector = embed_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vector, top_k)
    return [documents[i] for i in indices[0]]

# Test retrieval
print(retrieve_docs("How can I get funding?"))


['The government offers startup grants through various schemes.', 'Networking events connect entrepreneurs with investors and mentors.']


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load FLAN-T5 Small
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Generate response
def generate_response(query):
    retrieved_docs = retrieve_docs(query)
    context = " ".join(retrieved_docs)

    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test chatbot
print(generate_response("Tell me about startup funding."))


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Startup India provides funding and tax benefits for new startups in India.


In [ ]:
!git clone https://huggingface.co/spaces/YOUR_USERNAME/startup-chatbot
%cd startup-chatbot



Cloning into 'startup-chatbot'...
fatal: could not read Username for 'https://huggingface.co': No such device or address
[Errno 2] No such file or directory: 'startup-chatbot'
/content


In [ ]:
%%writefile app.py
from fastapi import FastAPI
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import faiss
import torch
from sentence_transformers import SentenceTransformer

app = FastAPI()

# Load models
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Sample documents
documents = [
    "Startup India provides funding and tax benefits for new startups in India.",
    "Angel investors are individuals who invest in early-stage startups in exchange for equity.",
    "A pitch deck is a presentation that startups use to attract investors.",
    "The government offers startup grants through various schemes.",
    "Networking events connect entrepreneurs with investors and mentors."
]

# Convert documents to embeddings and store in FAISS
doc_vectors = embed_model.encode(documents, convert_to_numpy=True)
dimension = doc_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_vectors)

def retrieve_docs(query, top_k=2):
    query_vector = embed_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vector, top_k)
    return [documents[i] for i in indices[0]]

def generate_response(query):
    retrieved_docs = retrieve_docs(query)
    context = " ".join(retrieved_docs)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

@app.get("/chat")
def chat(query: str):
    return {"response": generate_response(query)}


Writing app.py


In [ ]:
%%writefile requirements.txt
faiss-cpu
transformers
sentence-transformers
fastapi
uvicorn
torch


Writing requirements.txt


In [ ]:
%%writefile Dockerfile
# Use official Python image
FROM python:3.10

# Set working directory
WORKDIR /app

# Copy files
COPY . /app

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Expose API port
EXPOSE 7860

# Start FastAPI server
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "7860"]


Writing Dockerfile


In [ ]:
!git add .
!git commit -m "Deploy startup chatbot API"
!git push


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
!git clone https://huggingface.co/spaces/key-life/startup-chatbot

Cloning into 'startup-chatbot'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (13/13), 5.06 KiB | 1.01 MiB/s, done.


In [ ]:
!git clone https://huggingface.co/spaces/key-life/VentureAI

Cloning into 'VentureAI'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9 (delta 1), reused 1 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (9/9), 3.21 KiB | 1.07 MiB/s, done.


In [ ]:
import shutil
import os

# Define source and destination
source_files = ["/content/main.py", "/content/app.py", "/content/requirements.txt"]
destination_folder = "/content/VentureAI/"

# Move files if they exist, handling existing files
for file in source_files:
    destination_path = os.path.join(destination_folder, os.path.basename(file))
    if os.path.exists(file) and not os.path.exists(destination_path): # Check if dest file exists
        shutil.move(file, destination_folder)
    elif os.path.exists(destination_path):
        print(f"Warning: {destination_path} already exists! Skipping.") # Notify if file exists
    else:
        print(f"Warning: {file} not found!")


In [ ]:
!ls -R /content


/content:
app.py	Dockerfile  requirements.txt  sample_data  startup-chatbot  VentureAI

/content/sample_data:
anscombe.json		     california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv		   README.md

/content/startup-chatbot:
app.py	Dockerfile  README.md  requirements.txt

/content/VentureAI:
app.py	README.md  requirements.txt


In [ ]:
!find /content -name "main.py"


In [ ]:
with open("/content/VentureAI/main.py", "w") as f:
    f.write("# This is the main.py file for VentureAI chatbot\n")
print("main.py has been created!")


main.py has been created!


In [ ]:
cd /content/VentureAI

/content/VentureAI


In [ ]:
!git config --global user.email "sejalqp@gmail.com"
!git config --global user.name "key-life"


In [ ]:
!git add .
!git commit -m "Added main.py and updated chatbot files"
!git push

[main 8e4d88c] Added main.py and updated chatbot files
 1 file changed, 1 insertion(+)
 create mode 100644 main.py
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 326 bytes | 326.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/spaces/key-life/VentureAI
   d32f93a..8e4d88c  main -> main
